In [1]:
import calendar

In [1]:
from datetime import datetime, timedelta

from dateutil.relativedelta import relativedelta


def same_day_last_year(date):
    return date - relativedelta(years=1)


# Example usage:
today = datetime(2023, 2, 15)
same_day_last_year = same_day_last_year(today)
print(same_day_last_year)  # Output: 2022-02-16 00:00:00

2022-02-15 00:00:00


In [ ]:
def _create_period_boundaries(self, periods, calendar_type):
    prefix = "fiscal_" if calendar_type == "fiscal" else ""
    boundaries = {}

    # Fiscal only needs year/quarter boundaries
    boundary_keys = (
        ["year", "quarter"]
        if calendar_type == "fiscal"
        else ["year", "quarter", "month", "week"]
    )

    for key in boundary_keys:
        period_key = (
            f"{prefix}period_{key}" if calendar_type == "fiscal" else f"period_{key}"
        )
        boundaries.update(
            {
                f"start_of_{prefix}{key}": periods[period_key].dt.start_time,
                f"end_of_{prefix}{key}": periods[period_key].dt.end_time.dt.floor("D"),
            }
        )
    return boundaries

In [ ]:
import datetime

import pandas as pd
from django.db.models import Max, Min


class CalendarGenerator:
    def __init__(self):
        self.date_info = self.get_date_info()
        self.today = datetime.date.today()
        self.current_week_start = self.get_current_week_start()
        self.start_current_fy = self.get_start_current_fy()

    def get_date_info(self):
        return BookingsFinancialData.objects.aggregate(
            min_check_in=Min("check_in"),
            max_check_in=Max("check_in"),
            max_booking=Max("booking_date"),
        )

    def get_current_week_start(self):
        return self.today - datetime.timedelta(days=self.today.isoweekday() - 1)

    def get_start_current_fy(self):
        return (
            datetime.date(self.today.year, 3, 1)
            if self.today.month >= 3
            else datetime.date(self.today.year - 1, 3, 1)
        )

    def generate_calendar(self):
        if not self.date_info["min_check_in"] or not self.date_info["max_check_in"]:
            return pd.DataFrame()

        start_date, end_date = self.get_date_range()
        df = self.create_date_dataframe(start_date, end_date)

        self.add_date_columns(df)
        self.add_week_columns(df)
        self.add_month_columns(df)
        self.add_offset_columns(df)
        self.add_fiscal_year_columns(df)

        return df

    def get_date_range(self):
        start_year = self.date_info["min_check_in"].year
        end_year = self.date_info["max_check_in"].year
        start_date = datetime.date(start_year, 1, 1)
        end_date = datetime.date(end_year, 12, 31)
        return start_date, end_date

    def create_date_dataframe(self, start_date, end_date):
        df = pd.DataFrame({"date": pd.date_range(start=start_date, end=end_date)})
        df["date"] = pd.to_datetime(df["date"])
        return df

    def add_date_columns(self, df):
        df["calendar_year_number"] = df["date"].dt.year
        df["calendar_year"] = "CY " + df["calendar_year_number"].astype(str)
        df["year_month"] = df["date"].dt.strftime("%b %Y")
        df["year_month_number"] = df["calendar_year_number"] * 12 + (
            df["date"].dt.month - 1
        )
        df["month"] = df["date"].dt.month_name().str[:3]
        df["month_number"] = df["date"].dt.month

    def add_week_columns(self, df):
        iso_week = df["date"].dt.isocalendar()
        df["week_number"] = iso_week.week
        df["weekday_number"] = df["date"].dt.weekday + 1
        df["weekday"] = df["date"].dt.day_name().str[:3]
        df["start_of_week"] = df["date"] - pd.to_timedelta(
            df["weekday_number"] - 1, unit="D"
        )
        df["end_of_week"] = df["start_of_week"] + pd.to_timedelta(6, unit="D")
        df["is_weekend"] = df["weekday_number"].isin([6, 7])

    def add_month_columns(self, df):
        df["start_of_month"] = df["date"].dt.to_period("M").dt.start_time
        df["end_of_month"] = df["date"].dt.to_period("M").dt.end_time

    def add_offset_columns(self, df):
        df["same_day_last_year"] = df["date"] - pd.DateOffset(days=364)
        current_week_start_dt = pd.to_datetime(self.current_week_start)
        df["week_offset"] = (df["start_of_week"] - current_week_start_dt).dt.days // 7

    def add_fiscal_year_columns(self, df):
        start_fy_dt = pd.to_datetime(self.start_current_fy)
        end_fy_dt = start_fy_dt + pd.DateOffset(years=1)
        df["is_current_fiscal_year"] = (df["date"] >= start_fy_dt) & (
            df["date"] < end_fy_dt
        )

        if self.date_info["max_booking"]:
            last_txn_dt = pd.to_datetime(self.date_info["max_booking"])
            df["with_transaction"] = df["date"].dt.date <= last_txn_dt.date()
        else:
            df["with_transaction"] = False

        date_cols = [
            "date",
            "start_of_week",
            "end_of_week",
            "start_of_month",
            "end_of_month",
            "same_day_last_year",
        ]
        for col in date_cols:
            df[col] = df[col].dt.date

In [13]:
from collections import defaultdict

import pandas as pd


def calculate_portfolio(transactions, market_prices=None, sell_fee_rate=0.00895):
    """
    Track both realized and unrealized P/L with sell fees
    """
    holdings = defaultdict(
        lambda: {
            "quantity": 0,
            "cost_basis": 0.0,
            "realized_pl": 0.0,
            "total_buy": 0.0,
            "total_sell": 0.0,
        }
    )

    # Process all transactions
    for transaction in transactions:
        _, trans_type, security, qty, price, fees, total = transaction

        if trans_type == "Buy":
            holdings[security]["quantity"] += qty
            holdings[security]["cost_basis"] += total
            holdings[security]["total_buy"] += total

        elif trans_type == "Sell":
            # Calculate cost basis of sold shares
            avg_cost = holdings[security]["cost_basis"] / holdings[security]["quantity"]
            sold_cost = avg_cost * qty

            # Update realized P/L (Total already includes fees)
            holdings[security]["realized_pl"] += total - sold_cost

            # Update holdings
            holdings[security]["quantity"] -= qty
            holdings[security]["cost_basis"] -= sold_cost
            holdings[security]["total_sell"] += total

    # Build portfolio summary
    portfolio = []
    for security, data in holdings.items():
        if data["quantity"] > 0 or data["realized_pl"] != 0:
            current_price = market_prices.get(security, 0)
            avg_cost = (
                (data["cost_basis"] / data["quantity"]) if data["quantity"] > 0 else 0
            )

            # Calculate unrealized P/L with sell fees
            gross_value = data["quantity"] * current_price
            estimated_fee = gross_value * sell_fee_rate
            net_market_value = gross_value - estimated_fee
            unrealized_pl = net_market_value - data["cost_basis"]

            portfolio.append(
                {
                    "Security": security,
                    "Quantity": data["quantity"],
                    "Avg Cost": (round(avg_cost, 4) if data["quantity"] > 0 else 0),
                    "Realized P/L": round(data["realized_pl"], 2),
                    "Unrealized P/L": round(unrealized_pl, 2),
                    "Total P/L": round(data["realized_pl"] + unrealized_pl, 2),
                    "Market Value": round(net_market_value, 2),
                }
            )

    return pd.DataFrame(portfolio)


# Market prices and transactions (same as previous)
market_prices = {"DITO": 1.74, "ICT": 355.00, "MBT": 72.25, "SGP": 12.00}

# Generate report
portfolio_df = calculate_portfolio(transactions, market_prices)
display(portfolio_df.set_index("Security"))

ValueError: not enough values to unpack (expected 7, got 4)

In [12]:
import numpy as np
import pandas as pd


def calculate_portfolio_metrics(transactions, current_prices):
    portfolio = {}
    for security in transactions["Security"].unique():
        security_txns = transactions[transactions["Security"] == security]

        quantity = 0
        total_cost = 0
        realized_pl = 0

        for _, txn in security_txns.iterrows():
            if txn["Transaction"] == "Buy":
                quantity += txn["Quantity"]
                total_cost += txn["Total"]
            elif txn["Transaction"] == "Sell":
                avg_cost = total_cost / quantity
                realized_pl += txn["Total"] - (txn["Quantity"] * avg_cost)
                quantity -= txn["Quantity"]
                total_cost -= txn["Quantity"] * avg_cost

        if quantity > 0:
            avg_cost = total_cost / quantity
            current_price = current_prices[security]
            market_value = (
                quantity * current_price * (1 - 0.00895)
            )  # Subtracting sell fee
            unrealized_pl = market_value - total_cost

            portfolio[security] = {
                "Quantity": quantity,
                "Average Cost": avg_cost,
                "Current Price": current_price,
                "Market Value": market_value,
                "Unrealized P/L": unrealized_pl,
                "Realized P/L": realized_pl,
            }

    df = pd.DataFrame(portfolio).T
    df["% of Portfolio"] = df["Market Value"] / df["Market Value"].sum() * 100

    total_market_value = df["Market Value"].sum()
    total_cost = df["Quantity"] * df["Average Cost"]
    total_unrealized_pl = df["Unrealized P/L"].sum()
    total_realized_pl = df["Realized P/L"].sum()

    return df, total_market_value, total_unrealized_pl, total_realized_pl


# Example usage
transactions = pd.DataFrame(
    [
        {
            "Date": "2025-02-07",
            "Transaction": "Buy",
            "Security": "SGP",
            "Quantity": 1000,
            "Price": 11.32,
            "Fees": 33.3940,
            "Total": 11353.3940,
        },
        {
            "Date": "2025-02-07",
            "Transaction": "Buy",
            "Security": "DITO",
            "Quantity": 6000,
            "Price": 1.75,
            "Fees": 30.9750,
            "Total": 10530.9750,
        },
        {
            "Date": "2025-02-07",
            "Transaction": "Buy",
            "Security": "MBT",
            "Quantity": 200,
            "Price": 73.00,
            "Fees": 43.0700,
            "Total": 14643.0700,
        },
        {
            "Date": "2025-02-07",
            "Transaction": "Buy",
            "Security": "ICT",
            "Quantity": 30,
            "Price": 361.00,
            "Fees": 31.9485,
            "Total": 10861.9485,
        },
        {
            "Date": "2025-02-12",
            "Transaction": "Buy",
            "Security": "ICT",
            "Quantity": 30,
            "Price": 332.00,
            "Fees": 29.3820,
            "Total": 9989.3820,
        },
        {
            "Date": "2025-02-12",
            "Transaction": "Buy",
            "Security": "DITO",
            "Quantity": 6000,
            "Price": 1.73,
            "Fees": 30.6210,
            "Total": 10410.6210,
        },
        {
            "Date": "2025-02-12",
            "Transaction": "Buy",
            "Security": "MBT",
            "Quantity": 200,
            "Price": 71.45,
            "Fees": 42.1555,
            "Total": 14332.1555,
        },
        {
            "Date": "2025-02-12",
            "Transaction": "Buy",
            "Security": "SGP",
            "Quantity": 1000,
            "Price": 11.54,
            "Fees": 34.0430,
            "Total": 11574.0430,
        },
        {
            "Date": "2025-02-13",
            "Transaction": "Sell",
            "Security": "SGP",
            "Quantity": 1000,
            "Price": 11.86,
            "Fees": 106.1470,
            "Total": 11753.8530,
        },
        {
            "Date": "2025-02-13",
            "Transaction": "Buy",
            "Security": "SGP",
            "Quantity": 1000,
            "Price": 12.26,
            "Fees": 36.1670,
            "Total": 12296.1670,
        },
    ]
)

current_prices = {"DITO": 1.74, "ICT": 355.00, "MBT": 72.25, "SGP": 12.00}

portfolio_df, total_value, total_unrealized_pl, total_realized_pl = (
    calculate_portfolio_metrics(transactions, current_prices)
)

display(portfolio_df)
print(f"\nTotal Portfolio Value: ₱{total_value:.2f}")
print(f"Total Unrealized P/L: ₱{total_unrealized_pl:.2f}")
print(f"Total Realized P/L: ₱{total_realized_pl:.2f}")

,Quantity,Average Cost,Current Price,Market Value,Unrealized P/L,Realized P/L,% of Portfolio
SGP,2000.0,11.879943,12.00,23785.200,25.3145,290.1345,25.241902
DITO,12000.0,1.745133,1.74,20693.124,-248.4720,0.0000,21.960454
MBT,400.0,72.438064,72.25,28641.345,-333.8805,0.0000,30.395456
ICT,60.0,347.522175,355.00,21109.365,258.0345,0.0000,22.402188



Total Portfolio Value: ₱94229.03
Total Unrealized P/L: ₱-299.00
Total Realized P/L: ₱290.13


In [16]:
from collections import defaultdict

import pandas as pd

# Sample transactions with CORRECT structure (7 elements per record)
transactions = [
    ("2025-02-07", "Buy", "SGP", 1000, 11.32, 33.3940, 11353.3940),
    ("2025-02-07", "Buy", "DITO", 6000, 1.75, 30.9750, 10530.9750),
    ("2025-02-07", "Buy", "MBT", 200, 73.00, 43.0700, 14643.0700),
    ("2025-02-07", "Buy", "ICT", 30, 361.00, 31.9485, 10861.9485),
    ("2025-02-12", "Buy", "ICT", 30, 332.00, 29.3820, 9989.3820),
    ("2025-02-12", "Buy", "DITO", 6000, 1.73, 30.6210, 10410.6210),
    ("2025-02-12", "Buy", "MBT", 200, 71.45, 42.1555, 14332.1555),
    ("2025-02-12", "Buy", "SGP", 1000, 11.54, 34.0430, 11574.0430),
    ("2025-02-13", "Sell", "SGP", 1000, 11.86, 106.1470, 11753.8530),
    ("2025-02-13", "Buy", "SGP", 1000, 12.26, 36.1670, 12296.1670),
]


def calculate_portfolio(transactions, market_prices=None, sell_fee_rate=0.00895):
    holdings = defaultdict(
        lambda: {"quantity": 0, "cost_basis": 0.0, "realized_pl": 0.0}
    )

    for record in transactions:
        # Unpack all 7 values from each transaction record
        date, trans_type, security, qty, price, fees, total = record

        if trans_type == "Buy":
            holdings[security]["quantity"] += qty
            holdings[security]["cost_basis"] += total

        elif trans_type == "Sell":
            if holdings[security]["quantity"] > 0:
                avg_cost = (
                    holdings[security]["cost_basis"] / holdings[security]["quantity"]
                )
                sold_cost = avg_cost * qty

                holdings[security]["realized_pl"] += total - sold_cost
                holdings[security]["quantity"] -= qty
                holdings[security]["cost_basis"] -= sold_cost

    # Generate portfolio report
    portfolio = []
    for security, data in holdings.items():
        if data["quantity"] > 0 or data["realized_pl"] != 0:
            current_price = market_prices.get(security, 0) if market_prices else 0
            avg_cost = (
                (data["cost_basis"] / data["quantity"]) if data["quantity"] > 0 else 0
            )

            # Calculate market value with fees
            gross_value = data["quantity"] * current_price
            estimated_fee = gross_value * sell_fee_rate
            net_value = gross_value - estimated_fee

            portfolio.append(
                {
                    "Security": security,
                    "Quantity": data["quantity"],
                    "Avg Cost": round(avg_cost, 4),
                    "Realized P/L": round(data["realized_pl"], 2),
                    "Unrealized P/L": round(net_value - data["cost_basis"], 2),
                    "Total Value": round(net_value, 2),
                }
            )

    return pd.DataFrame(portfolio)


# Market prices
market_prices = {"DITO": 1.74, "ICT": 355.00, "MBT": 72.25, "SGP": 12.00}

# Generate and display portfolio
portfolio_df = calculate_portfolio(transactions, market_prices)
display(portfolio_df.set_index("Security"))

,Quantity,Avg Cost,Realized P/L,Unrealized P/L,Total Value
Security,,,,,
SGP,2000,11.8799,290.13,25.31,23785.20
DITO,12000,1.7451,0.00,-248.47,20693.12
MBT,400,72.4381,0.00,-333.88,28641.35
ICT,60,347.5222,0.00,258.03,21109.37


In [21]:
# Current market prices
market_prices = {"SGP": 12.00, "DITO": 1.74, "ICT": 355.00, "MBT": 72.25}

# Track portfolio data (using previous calculations)
portfolio = []
total_cost = 0
total_market_value = 0

for security, data in security_data.items():
    qty = data["total_quantity"]
    avg_cost = data["total_cost"] / qty if qty != 0 else 0
    current_price = market_prices.get(security, 0)
    cost_basis = qty * avg_cost
    market_value = qty * current_price
    unrealized = market_value - cost_basis
    pnl_pct = (unrealized / cost_basis * 100) if cost_basis != 0 else 0

    portfolio.append(
        {
            "Security": security,
            "Quantity": qty,
            "Avg Cost": avg_cost,
            "Current Price": current_price,
            "Total Cost": cost_basis,
            "Market Value": market_value,
            "Unrealized P&L": unrealized,
            "P&L %": pnl_pct,
        }
    )

    total_cost += cost_basis
    total_market_value += market_value

# Add totals row
portfolio.append(
    {
        "Security": "Total",
        "Total Cost": total_cost,
        "Market Value": total_market_value,
        "Unrealized P&L": total_market_value - total_cost,
        "P&L %": (
            ((total_market_value - total_cost) / total_cost * 100)
            if total_cost != 0
            else 0
        ),
    }
)

# Print formatted table
print(
    f"{'Security':<8} {'Quantity':>8} {'Avg Cost':>10} {'Current Price':>12} {'Total Cost':>12} {'Market Value':>12} {'Unrealized P&L':>14} {'P&L %':>10}"
)
for item in portfolio:
    if item["Security"] == "Total":
        print(
            f"{item['Security']:<8} {'':>8} {'':>10} {'':>12} ${item['Total Cost']:>11.2f} ${item['Market Value']:>11.2f} ${item['Unrealized P&L']:>13.2f} {item['P&L %']:>10.2f}%"
        )
    else:
        print(
            f"{item['Security']:<8} {item['Quantity']:>8,} ${item['Avg Cost']:>9.4f} ${item['Current Price']:>11.2f} ${item['Total Cost']:>11.2f} ${item['Market Value']:>11.2f} ${item['Unrealized P&L']:>13.2f} {item['P&L %']:>9.2f}%"
        )

Security Quantity   Avg Cost Current Price   Total Cost Market Value Unrealized P&L      P&L %
SGP         2,000 $  11.8799 $      12.00 $   23759.89 $   24000.00 $       240.11      1.01%
DITO       12,000 $   1.7451 $       1.74 $   20941.60 $   20880.00 $       -61.60     -0.29%
MBT           400 $  72.4381 $      72.25 $   28975.23 $   28900.00 $       -75.23     -0.26%
ICT            60 $ 347.5222 $     355.00 $   20851.33 $   21300.00 $       448.67      2.15%
Total                                     $   94528.04 $   95080.00 $       551.96       0.58%


In [22]:
from collections import defaultdict

import pandas as pd

# Sample transactions with CORRECT structure (7 elements per record)
transactions = [
    ("2025-02-07", "Buy", "SGP", 1000, 11.32, 33.3940, 11353.3940),
    ("2025-02-07", "Buy", "DITO", 6000, 1.75, 30.9750, 10530.9750),
    ("2025-02-07", "Buy", "MBT", 200, 73.00, 43.0700, 14643.0700),
    ("2025-02-07", "Buy", "ICT", 30, 361.00, 31.9485, 10861.9485),
    ("2025-02-12", "Buy", "ICT", 30, 332.00, 29.3820, 9989.3820),
    ("2025-02-12", "Buy", "DITO", 6000, 1.73, 30.6210, 10410.6210),
    ("2025-02-12", "Buy", "MBT", 200, 71.45, 42.1555, 14332.1555),
    ("2025-02-12", "Buy", "SGP", 1000, 11.54, 34.0430, 11574.0430),
    ("2025-02-13", "Sell", "SGP", 1000, 11.86, 106.1470, 11753.8530),
    ("2025-02-13", "Buy", "SGP", 1000, 12.26, 36.1670, 12296.1670),
]


def calculate_portfolio(transactions, market_prices=None, sell_fee_rate=0.00895):
    holdings = defaultdict(
        lambda: {"quantity": 0, "cost_basis": 0.0, "realized_pl": 0.0}
    )

    for record in transactions:
        # Unpack all 7 values from each transaction record
        date, trans_type, security, qty, price, fees, total = record

        if trans_type == "Buy":
            holdings[security]["quantity"] += qty
            holdings[security]["cost_basis"] += total

        elif trans_type == "Sell":
            if holdings[security]["quantity"] > 0:
                avg_cost = (
                    holdings[security]["cost_basis"] / holdings[security]["quantity"]
                )
                sold_cost = avg_cost * qty

                holdings[security]["realized_pl"] += total - sold_cost
                holdings[security]["quantity"] -= qty
                holdings[security]["cost_basis"] -= sold_cost

    # Generate portfolio report
    portfolio = []
    for security, data in holdings.items():
        if data["quantity"] > 0 or data["realized_pl"] != 0:
            current_price = market_prices.get(security, 0) if market_prices else 0
            avg_cost = (
                (data["cost_basis"] / data["quantity"]) if data["quantity"] > 0 else 0
            )

            # Calculate market value with fees
            gross_value = data["quantity"] * current_price
            estimated_fee = gross_value * sell_fee_rate
            net_value = gross_value - estimated_fee

            portfolio.append(
                {
                    "Security": security,
                    "Quantity": data["quantity"],
                    "Avg Cost": round(avg_cost, 4),
                    "Realized P/L": round(data["realized_pl"], 2),
                    "Unrealized P/L": round(net_value - data["cost_basis"], 2),
                    "Total Value": round(net_value, 2),
                }
            )

    return pd.DataFrame(portfolio)


# Market prices
market_prices = {"DITO": 1.74, "ICT": 355.00, "MBT": 72.25, "SGP": 12.00}

# Generate and display portfolio
portfolio_df = calculate_portfolio(transactions, market_prices)
display(portfolio_df.set_index("Security"))

,Quantity,Avg Cost,Realized P/L,Unrealized P/L,Total Value
Security,,,,,
SGP,2000,11.8799,290.13,25.31,23785.20
DITO,12000,1.7451,0.00,-248.47,20693.12
MBT,400,72.4381,0.00,-333.88,28641.35
ICT,60,347.5222,0.00,258.03,21109.37


In [28]:
import pandas as pd

# Sample transactions (as tuples)
transactions = [
    ("2025-02-07", "Buy", "SGP", 1000, 11.32, 33.394, 11353.394),
    ("2025-02-07", "Buy", "DITO", 6000, 1.75, 30.975, 10530.975),
    ("2025-02-07", "Buy", "MBT", 200, 73.00, 43.070, 14643.070),
    ("2025-02-07", "Buy", "ICT", 30, 361.00, 31.9485, 10861.9485),
    ("2025-02-12", "Buy", "ICT", 30, 332.00, 29.382, 9989.382),
    ("2025-02-12", "Buy", "DITO", 6000, 1.73, 30.621, 10410.621),
    ("2025-02-12", "Buy", "MBT", 200, 71.45, 42.1555, 14332.1555),
    ("2025-02-12", "Buy", "SGP", 1000, 11.54, 34.043, 11574.043),
    ("2025-02-13", "Sell", "SGP", 1000, 11.86, 106.147, 11753.853),
    ("2025-02-13", "Buy", "SGP", 1000, 12.26, 36.167, 12296.167),
]

# Convert tuples to dictionaries for clarity
tx_list = []
for tx in transactions:
    tx_dict = {
        "Date": tx[0],
        "Transaction": tx[1],
        "Security": tx[2],
        "Quantity": tx[3],
        "Price": tx[4],
        "Fees": tx[5],
        "Total": tx[6],
    }
    tx_list.append(tx_dict)

# Track holdings and P&L
security_data = {}
market_prices = {"SGP": 12.00, "DITO": 1.74, "MBT": 72.25, "ICT": 355.00}

for tx in tx_list:
    sec = tx["Security"]
    action = tx["Transaction"]
    qty = tx["Quantity"]
    price = tx["Price"]
    fees = tx["Fees"]

    if sec not in security_data:
        security_data[sec] = {
            "total_qty": 0,
            "total_cost": 0.0,
            "realized_pnl": 0.0,
        }

    if action == "Buy":
        cost = (price * qty) + fees
        security_data[sec]["total_qty"] += qty
        security_data[sec]["total_cost"] += cost
    elif action == "Sell":
        avg_cost = security_data[sec]["total_cost"] / security_data[sec]["total_qty"]
        cost_basis = avg_cost * qty
        proceeds = (price * qty) - fees
        security_data[sec]["realized_pnl"] += proceeds - cost_basis
        security_data[sec]["total_qty"] -= qty
        security_data[sec]["total_cost"] -= cost_basis

# Build portfolio DataFrame
portfolio = []
for sec, data in security_data.items():
    qty = data["total_qty"]
    avg_cost = data["total_cost"] / qty if qty > 0 else 0
    current_price = market_prices.get(sec, 0)

    # Market value after 0.895% sell fee
    market_value = (qty * current_price) * (1 - 0.00895)

    portfolio.append(
        {
            "Security": sec,
            "Quantity": qty,
            "Avg Cost": avg_cost,
            "Current Price": current_price,
            "Total Cost": data["total_cost"],
            "Market Value": market_value,
            "Unrealized P&L": market_value - data["total_cost"],
            "Realized P&L": data["realized_pnl"],
        }
    )

# Create DataFrame and calculate totals
df = pd.DataFrame(portfolio)
df["Total P&L"] = df["Unrealized P&L"] + df["Realized P&L"]

# Add totals row
totals = df[
    [
        "Total Cost",
        "Market Value",
        "Unrealized P&L",
        "Realized P&L",
        "Total P&L",
    ]
].sum()
totals_df = pd.DataFrame([totals], index=["Total"])
totals_df.insert(0, "Security", "Total")

# Combine into final DataFrame
final_df = pd.concat([df, totals_df], ignore_index=True)

# Format numeric columns
numeric_cols = [
    "Avg Cost",
    "Current Price",
    "Total Cost",
    "Market Value",
    "Unrealized P&L",
    "Realized P&L",
    "Total P&L",
]
final_df[numeric_cols] = final_df[numeric_cols].round(2)

# Display results
display(final_df.set_index("Security"))

,Quantity,Avg Cost,Current Price,Total Cost,Market Value,Unrealized P&L,Realized P&L,Total P&L
Security,,,,,,,,
SGP,2000.0,11.88,12.00,23759.89,23785.20,25.31,290.13,315.45
DITO,12000.0,1.75,1.74,20941.60,20693.12,-248.47,0.00,-248.47
MBT,400.0,72.44,72.25,28975.23,28641.34,-333.88,0.00,-333.88
ICT,60.0,347.52,355.00,20851.33,21109.36,258.03,0.00,258.03
Total,NaN,NaN,NaN,94528.04,94229.03,-299.00,290.13,-8.87


In [ ]:
DISTRIBUTOR = {
    "MASTER_HOSTS": {
        "societies": "http://distributor.masters.api.pro.logitravel.internal/api/organizations/{}/products/agencies/societies",  # NOQA
        "society_agencies": "http://distributor.masters.api.pro.logitravel.internal/api/organizations/{}/products/agencies/societies/{}",  # NOQA
        "providers": "http://distributor.masters.api.pro.logitravel.internal/api/organizations/lgt/products/hot/providers/master",  # NOQA
        "clients_groups": "http://distributor.masters.api.pro.logitravel.internal/organizations/lgt/agencies/products/hot/getClientGroupList",  # NOQA
    }
}